# Chapter 9

## SQLite

In [39]:
using SQLite

In [40]:
db = SQLite.DB("quotes.db")   # SQLiteDB() etc., are not exported

SQLite.DB("quotes.db")

In [45]:
SQLite.tables(db)

,name
,String⍰
1,categories
2,quotes


In [46]:
SQLite.columns(db,"quotes")

,cid,name,type,notnull,dflt_value,pk
,Int64⍰,String⍰,String⍰,Int64⍰,Any,Int64⍰
1,0,id,integer,1,missing,1
2,1,cid,integer,1,missing,0
3,2,author,varchar(100),0,missing,0
4,3,quoname,varchar(250),1,missing,0


In [41]:
# New syntax, using dataframes

df = DataFrame(SQLite.Query(db, "select count(*) as N from quotes"))

,N
,Int64⍰
1,36


In [44]:
df[1]

1-element Array{Union{Missing, Int64},1}:
 36

In [14]:
df = DataFrame(SQLite.Query(db, "select * from quotes limit 10"))

,id,cid,author,quoname
,Int64⍰,Int64⍰,String⍰,String⍰
1,1,1,Hofstadter's Law,"It always takes longer than you expect, even when you take Hofstadter's Law into account."
2,2,2,Noelie Altito,The shortest distance between two points is under construction.
3,3,3,Scott's Law,Adding manpower to a late software project makes it later
4,4,2,Shaw's Principle,"Build a system that even a fool can use, and only a fool will want to use it."
5,5,4,Adolf Hiltler,The great mass of the people will more easily fall victims to a big lie than a small one
6,6,5,G. B. Shaw,There is no satisfaction in hanging a man who does not object to it
7,7,1,Heller's Law,The first myth of management is that it exists
8,8,3,missing,There are two ways to write error-free programs. Only the third one works.
9,9,2,Fingle's Creed,Science is true. Don't be misled by facts.


In [64]:
sql =  "select q.quoname, q.author, c.catname from quotes q ";
sql *= "join categories c on q.cid = c.id limit 5";

df = DataFrame(SQLite.Query(db,sql))

,quoname,author,catname
,String⍰,String⍰,String⍰
1,"It always takes longer than you expect, even when you take Hofstadter's Law into account.",Hofstadter's Law,Words of Wisdom
2,The shortest distance between two points is under construction.,Noelie Altito,Science
3,"Build a system that even a fool can use, and only a fool will want to use it.",Shaw's Principle,Science
4,The great mass of the people will more easily fall victims to a big lie than a small one,Adolf Hiltler,Politics
5,There is no satisfaction in hanging a man who does not object to it,G. B. Shaw,Books & Plays


In [66]:
# The feather package can be used to store snapshots of queries
using Feather

Feather.write("QuoView01.feather", df)

"QuoView01.feather"

In [67]:
# Retrieving fro a feather file is "lazy"
# That is the metadata is read but records are fetched when they are referenced
# So feather files can hold very large datasets with little memory overhead.

dfx = Feather.read("QuoView01.feather");
size(dfx)

(5, 3)

In [70]:
dfx[1,1]   # Get first quote

" It always takes longer than you expect, even when you take Hofstadter's Law into account."

---

In [31]:
sql =  "select q.quoname from quotes q ";
sql *= " where q.author = 'Oscar Wilde'";
SQLite.Query(db,sql) |> DataFrame

,quoname
,String⍰
1,The only way to get rid of a temptation is to yield to it.
2,"There is only one thing in the world worse than being talked about, and that is not being talked about"
3,"I am not at all cynical, I have merely got experience, which, however, is very much the same thing"
4,To love oneself is the beginning of a lifelong romance
5,"We are all in the gutter, but some of us are looking at the stars"
6,"London society is full of women of the very highest birth who have, of their own free choice, remained thirty-five for years"


---

In [53]:
sql =  "select q.quoname, q.author, c.catname from quotes q ";
sql *= "join categories c on q.cid = c.id";

df = DataFrame(SQLite.Query(db,sql));
nrows = size(df)[1];
println("Number of rows: $nrows" )

Number of rows: 31


---

## MySQL

_See the **Chinook.ipynb** notebook_


---

## Postgresql

In [1]:
# Assumes Postgres is running and the Chinook dataset loaded into a database 'chinnok'
using LibPQ, DataStreams
conn = LibPQ.Connection("dbname=chinook");

In [2]:
res = execute(conn, "SELECT count(*) FROM \"Album\"");  # Note escaping the quotes
Data.stream!(res, NamedTuple)

(count = Union{Missing, Int64}[306],)

In [10]:
qry = fetch!(NamedTuple, execute(conn, "SELECT count(*) as NA FROM \"Album\""));
qry[1][1]

306

In [5]:
qry = fetch!(NamedTuple, execute(conn, "SELECT * FROM \"MediaType\""))

(MediaTypeId = Union{Missing, Int32}[1, 2, 3, 4, 5], Name = Union{Missing, String}["MPEG audio file", "Protected AAC audio file", "Protected MPEG-4 video file", "Purchased AAC audio file", "AAC audio file"])

In [7]:
qry[:Name]

5-element Array{Union{Missing, String},1}:
 "MPEG audio file"            
 "Protected AAC audio file"   
 "Protected MPEG-4 video file"
 "Purchased AAC audio file"   
 "AAC audio file"             

In [49]:
qry.FirstName[1]

"Margaret"

In [58]:
sqlx = """select e."FirstName", e."LastName", count(i."InvoiceId") as "Sales"
 from "Employee" as e
 join "Customer" as c on e."EmployeeId" = c."SupportRepId"
 join "Invoice" as i on i."CustomerId" = c."CustomerId"
 group by e."EmployeeId"
""";

In [59]:
qry = fetch!(NamedTuple, execute(conn, sqlx))

(FirstName = Union{Missing, String}["Margaret", "Jane", "Steve"], LastName = Union{Missing, String}["Park", "Peacock", "Johnson"], Sales = Union{Missing, Int64}[70, 97, 84])

In [60]:
using Printf
for i in 1:length(qry)
    @printf("%s %s has %d sales\n",
        qry.FirstName[i],qry.LastName[i],qry.Sales[i])
end

Margaret Park has 70 sales
Jane Peacock has 97 sales
Steve Johnson has 84 sales


---